# Part-based R-CNNs for Fine-grained Category Detection  

## Abstract  

通过特定对象局部区域细微外观差异得到的局部的语义可以进行细粒度分类。用于姿态标准化的表示的方法已经提出，但由于对象检测的困难，通常在测试时假定边界框标注。我们提出了一个细粒度分类模型，通过利用由下至上区域提议计算的深度卷积特征来克服这些限制。我们的方法学习整体和局部检测器，在它们之间强制学习几何约束，并从姿态标准化表示预测细粒度类别。对 Caltech UCSD 鸟类数据集的实验证实，我们的方法在端到端评估中胜过了最先进的细粒度分类方法，而无需在测试时间使用边界框。  

## 1. Introduction  

由于相关类别中特定区域外观的细微差异，视觉细粒度分类问题可能非常具有挑战性。与常见的识别相比，细粒度分类旨在区分不同品种或物种或产品模型，并且必须以物体姿态为条件进行可靠识别。面部识别是细粒度识别的经典案例，值得注意的是最好的面部识别方法都会去发现面部特征点并从这些位置提取特征。  

因此，定位目标的局部区域对于建立目标之间的对应关系以及避免因为姿势和相机视角变化带来的影响是很重要的。已经有工作研究了基于部件的方法来解决这个问题。局部定位确实是许多姿态归一化表示方法的瓶颈。之前已经使用了 Poselet 和 DPM 方法再局部定位上取得了一定的成功;只有在测试时给出已知的边界框时，方法通常才能得到较好的局部定位。通过开发一种新颖的深度局部检测方案，我们提出了一种端到端的细粒度分类系统，该系统在测试时不需要给定对象边界框，并且可以实现与先前报道的方法相媲美的性能，这些方法需要测试时给出真实标注框来过滤误报的检测结果。  

卷积网络最近在 ImageNet 挑战上的成功激发了将深度卷积特征应用于相关图像分类和检测任务的进一步工作。Girshick 等人通过将 CNN 应用于一组自下而上的候选区域提议，突破了对象检测的性能，与以前的最佳方法相比，PASCAL 检测性能提高了 30％ 以上。OverFeat 提出使用 CNN 进行目标位置的回来来进行检测。但是，利用深度卷积特征的进展并不局限于基本的目标检测。在许多应用中，如细粒度识别，属性识别，姿态估计等，合理的预测需要准确的区域定位。  

特征学习已被用于细粒度识别和属性估计，但受限于局部区域的工程特征。DPD-DeCAF 使用 DeCAF 作为特征描述符，但依靠基于 HOG 的 DPM 进行局部定位。PANDA 学习了特定部分深度卷积网络，其位置取决于基于 HOG 的 poselet 模型。这些模型缺乏 R-CNN 检测的鲁棒性。在这项工作中，我们探索了一种统一的方法，它使用相同的深度卷积表示进行检测以及局部描述。  

我们推测自下而上区域提议方法取得的进展，如 selective search，可以提高使目标及局部检测的性能。正如我们后面显示的那样，Caltech-UCSD 鸟类数据集使用 selective search proposals 的平均召回率为 95％。  

在本文中，我们提出了一个局部定位模型，它通过利用自下而上 selective search proposals 上计算的深度卷积特征来克服先前的细粒度识别系统的局限性。我们的方法学习局部外观模型并强化局部之间的几何约束。我们的方法概述如图 1 所示。我们已经研究了不同的几何约束，包括在语义外观空间中以最近邻为条件的联合区域定位的非参数模型。用广泛使用的细粒度基准 Caltech-UCSD 鸟类数据集来评估我们的方法，并得到 state-of-the-art 结果。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/Fine-grained/1/1.jpg?raw=true) 

## 2.  Related work  

### 2.1 Part-based models for detection and pose localization  

以前的工作已经提出了对对象局部外观和位置的显式建模，以便更精确地识别和定位。从图像结构开始，通过 poselets 和相关工作，许多方法已经联合定位了一系列的几何相关的区域。可变形零件模型（DPM），直到最近还是 PASCAL 物体检测最先进的方法，在与整个对象边界框相结合的位置上添加附加学习滤波器的模型部分，允许部件以已学会的变形成本对 anchor 进行偏移。DPM 将这种方法用于强监督训练，其中区域位置在训练时进行标注。这些方法的局限性在于它们使用弱特征（通常是 HOG）。  

### 2.2 Fine-grained categorization  

最近，大量的计算机视觉研究集中在许多领域的细粒度分类问题，如动物品种或物种，植物物种和人造物体。  

有几种方法基于检测和提取物体某些部分的特征。Farrell 等人提出了一种使用 poselets 的姿态归一化表示法。可变形零件模型在也被用于局部定位。基于定位脸部基准特征点的工作，Liu等人提出了一种基于样本的几何方法来检测狗脸并从关键点提取高度局部化的图像特征以区分狗的品种。此外，Berg 等人通过学习每对关键点的描述符，学习了一组高度区分的中间特征。此外，还有文章通过强化姿态和子类一致性来扩展以前的非参数样本法。Yao 等人和Yang 等人研究了模板匹配方法以降低滑动窗口方法的成本。 Goring 等人最近的工作将局部标注从具有类似全局形状的对象转换为非参数局部检测。但是，所有这些 part-based 的方法在测试时都需要真实边界框来进行局部定位或关键点预测。  

Human-in-the-loop 的方法要求人们命名对象的属性，点击某些部分或标记最具区分性的区域以提高分类的准确性。基于分割的方法对于细粒度识别也是非常有效的。有方法使用 region-level 线索来推断前景分割 mask 并丢弃背景中的噪声视觉信息。Chai 等表明联合学习局部定位和前景分割可以有益于细粒度分类。与大多数先前的 part-based 的方法类似，这些方法需要真实边界框来初始化分割种子。相比之下，我们的工作目标是在测试时不知道真实标注框的情况下进行端到端的细粒度分类。我们的局部检测器使用基于自底向上区域提议的卷积特征，以及学习的非参数几何约束来更精确地定位对象区域，从而实现强大的细粒度分类。  

### 2.3  Convolutional networks  

近年来，卷积神经网络（CNN）已被纳入各种领域的许多视觉识别系统中。这些模型的强度至少有一部分在于它们能够从原始数据输入（例如图像像素）中学习辨别特征，这与作为初始预处理步骤计算图像上的手工设计特征的更传统的物体识别流水线相反。LeCun 及其同事最初将这些模型应用于数字识别和OCR，后来应用于通用对象识别任务，CNN 得到了推广。随着大型标签图像数据库和 GPU 实现的引入，这些网络已经成为通用对象分类中最准确的方法。  

最近，通用对象检测方法已经开始利用深度 CNN，并且胜过基于传统特性的任何竞争方法。 OverFeat 使用 CNN 在粗滑动窗口检测框架中回归对象位置。对我们工作特别有启发的是 R-CNN 方法，该方法利用区域提案框架中 CNN 的深度特征，在 PASCAL VOC 数据集上实现前所未有的目标检测结果。我们的方法通过将 R-CNN 应用于除了整个对象之外的局部检测来推广 R-CNN，我们的实证结果证明这对精确的细粒度识别是至关重要的。  

## 3.  Part-based R-CNNs  

虽然证明了 R-CNN 方法在通用物体检测任务（PASCAL VOC）上的有效性，但没有探索这种方法同时在定位和细粒度识别中的应用。因为我们的工作是在这个机制中运作的，所以我们扩展了 R-CNN 来检测物体并将它们的局部定位并对定位进行几何约束。关于感兴趣对象的单个语义部分位置的假设（例如，用于动物类的头部位置），对于倾向于出现在相对于这些粗略固定的位置中的微妙外观差异进行建模变得合理部分。  

在 R-CNN 方法中，对于特定对象类别，具有 CNN 特征描述符 $\phi (x)$ 的候选检测 x 被赋予 $w_0^T \phi (x)$，其中 $w_0$ 是对象类别的 SVM 学习向量的权重。在我们的方法中，我们假设一个强有力的监督设置，在训练时间，我们不仅为整个对象设定了真实标注框，而且对一组固定的局部区域也进行了标注 ${p_1,p_2,...,p_n}$。  

在给定这些局部标注的情况下，在训练时，所有对象及其每个区域都被视为独立的对象类别：我们在区域提议中提取的特征描述符上训练一对多线性 SVM，其中与标注框重叠超过 0.7 的 proposal 被标注为该区域的正例，并且与任何标注框区域重叠小于 0.3 的区域被标记为负样本。因此，对于单个对象类别，我们为整体和局部 ${p_1,p_2,...,p_n}$ 分别学习了 SVM 权重 $w_0$ 和 ${w_1,w_2,...,w_n}$。在测试时，对于每个 proposal，我们计算来自所有SVM 的分数。当然，这些分数并不包含任何有关物体及其局部几何约束的知识；例如，在没有任何额外限制的情况下，鸟头检测器可以在检测器检测区域之外被检测到。因此，我们的最终联合对象和局部假设是使用下面几节中详述的几何打分函数来计算的，它强化了姿势预测与训练时观察姿势的统计数据一致的直观理想属性。  

### 3.1 Geometric constraints  

设 $X = {x_0，x_1，...，x_n}$ 表示对象 $p_0$ 和 n 个局部区域 ${p_i}_{i=1}^n$ 的位置（边界框），它们在训练数据中被标注，但在测试时是未知的。我们的目标是在先前未见过的测试图像中推断出物体位置和局部区域位置。给定目标和局部区域的 R-CNN 权重 ${w_0，w_1，...，w_n}$，我们将有相应的检测器 ${d_0，d_1，...，d_n}$，其中每个检测器得分为 $d_i(x)= \sigma (w_i^T \phi (x))$，其中 $\sigma (\cdot)$ 是 sigmoid 函数，$\phi (x)$ 是在位置 x 处提取的 CNN 特征描述符。我们通过解决以下优化问题来推断对象和局部区域的联合配置：  

$X^* = arg \ max_X \ \Delta(X) \prod_{i=0}^n d_i(x_i)$

其中 $\Delta(X)$ 定义了边界框在联合配置上的评分函数。我们考虑并报告了几个配置评分函数 $\Delta$ 的定量结果，在以下段落中详细介绍。  

框约束。 对物体和零件进行局部化的一个直观的想法是考虑每个可能的物体窗口和物体内的所有窗口，并选择具有最高分数的窗口。 在这种情况下，我们定义评分函数  

**边界框约束**。对物体和局部区域进行定位的一个直观的想法是考虑每个可能的物体窗口和物体内的所有窗口，并选择具有最高分数的窗口。在这种情况下，我们定义评分函数  

$\Delta_{box}(X) = \prod_{i=0}^n c_{x_0}(x_i)$

其中  

$ c_x(y) = \begin{cases} 1,&\text{if region y falls outside region x by at most $\epsilon$ pixels} \\ 0,&\text{otherwise} \end{cases} $

在我们的实验中，$\epsilon = 10$。  

**几何约束**。由于单个的区域检测器不完善，单个区域检测器得分最高的窗口并不总是正确的，特别是在有遮挡的情况下。因此，我们考虑几个评分函数，以限制检测结果相对于目标位置的布局从而过滤掉不正确的结果。我们定义  

$\Delta_{geometric}(X) = \Delta_{box}(X) \prod_{i=0}^n \delta_i (x_i)$

其中 $\delta_i$ 是给定训练数据的区域 $p_i$ 的位置的评分函数。在之前关于区域定位的工作中，我们试验了$\delta$的三个定义：  

- $\delta_i^{MG}(x_i)$ 将训练数据中区域 $p_i$ 的 $N_g$ 个成分拟合一个高斯混合模型。实验中，我们设定 $N_g = 4$。  

- $\delta_i^{NP}(x_i)$ 找出 $\tilde{x}_0$ 相似的 K 个最近邻，其中 $\tilde{x}_0 = arg \ max d_0(x_0)$ 是检测器检测结果评分最高的窗口，然后将这 K 个 窗口拟合一个高斯模型。实验中，设置 K = 20。图 2 展示了最近邻的一些例子。  

DPM 用每个分量的高斯先验来模拟 deformation costs。 R-CNN 是一个单成分模型，需要定义$\delta^{MG}$ 或 $\delta^{NP}$。我们的 $\delta^{NP}$ 定义受 Belhumeur 等人的启发，但不同之处在于我们在外观上而不是几何上定义最近邻。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/Fine-grained/1/2.jpg?raw=true)

### 3.2 Fine-grained categorization  

我们从局部区域以及整个对象中提取语义特征。最终的特征表示是 $ [\phi (x_0),...,\phi (x_n)] $ 其中 $x_0$ 和 $x_{1...n}$ 是使用上一节中的模型推导的整体和局部检测结果，$ \phi (x_i) $ 是区域 $x_i$ 的特征表示。  

在一组实验中，我们从 ImageNet 预先训练的 CNN 中提取深卷积特征 $ \phi (x_i) $，类似于 DeCAF。为了使 CNN 特征在细粒度鸟类分类的目标任务具有更强去区分性，我们还将从 CUB 数据集根基标注框切分的 200 类鸟类分类任务对 预训练的 CNN 网络进行 fune-tune。特别是，我们用一个新的 200 路 fc8 分层替换了原始的 1000 路 fc8 分类层，并随机初始化这层的权重。我们根据 R-CNN 的建议设置微调的学习率，将全局速率初始化为初始ImageNet 学习速率的十分之一，并在整个训练过程中将其降低 10 倍，但在新的 fc8 中学习速率这是全局学习率的 10 倍。对于整个对象边界框和每个区域边界框，我们独立地对 ImageNet 预先训练的 CNN 进行微调，以对每个区域进行分类，对每个区域进行调整为 227×227 网络输入大小。测试时，我们使用针对特定整个对象或局部区域进行微调的网络，为预测的整个对象或局部区域提取特征。  

为了对分类器进行训练，我们采用了一个一对多的线性 SVM 来使用最终的特征表示。对于新的测试图像，我们将全部和局部检测器与几何打分函数一起应用，以获取检测到的局部区域位置并使用特征进行预测。如果在测试图像中没有检测到特定区域 i（由于所有提议低于局部检测器的阈值），我们设置其特征 $ \phi (x_i) = 0$（零矢量）。  

## 4. Evaluation  

在本节中，我们对提出的方法进行了评估。具体而言，我们在广泛使用的细粒度基准 Caltech-UCSD 鸟类数据集上进行了实验。分类任务是区分 200 种鸟类，并且由于类别之间的高度相似性从而对计算机视觉系统具有挑战性。它包含 200 种鸟类的 11,788 幅图像。每个图像都用边界框和 15 个关键点的图像坐标标注：喙，背部，胸部，腹部，额头，冠，左眼，左腿，左翼，右眼，右腿，右翼，尾巴，颈背 和喉咙。我们对包含在数据集中的分组进行训练和测试，其中每个物种约 30 个训练样本。遵循之前的协议，我们使用鸟类数据集的两个语义部分：头部和身体。  

我们使用开源软件包 Caffe 来提取深度特征并微调我们的 CNN。对于目标和局部区域检测，我们使用 Caffe 参考模型，它与 Krizhevsky 等人使用的模型几乎相同。我们将 CNN 第 n 层的深层特征称为 convn，pooln 或 fcn，分别是卷积层，汇聚层或完全连接层的输出。我们使用 fc6 来训练 R-CNN 目标和局部区域检测器以及用于分类的图像表示。对于 $\delta^{NP}$，使用 pool5 和余弦距离度量来计算最近邻。  

### 4.1 Fine-grained categorization  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/Fine-grained/1/3.jpg?raw=true)

我们首先展示 Caltech-UCSD 鸟类数据集的标准细粒度分类任务上的结果。表 1 中的第一组结果是在测试时整个鸟类的真实标注框已知的情况下实现的，因为大多数 state-of-art 方法假设这样会使分类任务更容易一些。在这种情况下，我们具有局部非参数几何约束 $\delta^{NP}$ 的 part-based 方法在没有微调的情况下效果最好，无需微调即可实现 68.1％ 的分类精度。 Finetuning 大幅改善了这一结果，达到 76％ 以上。我们将结果与三种最先进的基准方法进行比较。我们使用深度卷积特征，但他们使用基于 HOG 的 DPM 作为其局部定位方法。性能的提高可能是由于更好的局部区域定位（见表4）。Oracle 方法在训练和测试时都使用真实的整体和局部标注。  

第二组是在测试时未知鸟的标注框的结果。大多数的文章都没有展示在这个更真实、更复杂数据集上的效果。如表 1 所示，在此设置中，我们 part-based 的方法比基准 DPD 模型效果更好。我们在不进行微调的情况下获得 66.0％ 的分类精度，几乎与给出真实边界框时的精度一样好。这意味着在测试时不需要对任何目标进行标注就可以进行分类。通过微调 CNN 模型，我们的方法分类准确率达到 73.89％。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/Fine-grained/1/4.jpg?raw=true)

我们做的另一个有趣的实验是移除局部区域描述符仅使用预测框中的整体图像描述符。通过几何约束，我们的方法能够帮助局部区域定位。如表 2 所示，我们的方法优于使用 R-CNN 的单个对象检测器，这意味着几何约束有助于我们的方法更好地定位对象窗口。强 DPM 的检测不如我们的方法准确，这就解释了它性能下降的原因。oracle 方法使用真实边界框，并且达到了 57.94％ 的准确率，这仍然远低于表 1 中在对象和局部区域内使用两个图像描述符的方法。  

### 4.2 Part localization  

我们现在展示我们系统在定位局部区域上的效果。在表 4 中的结果是根据正确定位的区域分（PCP）度量的百分比给出的。对于第一组结果，给出了整个对象边界框，任务仅仅是定位边界框中局部区域的位置，与真实局部区域标注相交面积大于 0.5 的认为是正确的结果。  

对于第二组结果，使用在第二部分中描述的目标函数在排名最高的局部区域预测上计算 PCP 度量。请注意，在这个更真实的设置中，我们不会假设在测试时就知道真实的目标标注框。  

如表 4 所示，对于给定边界框和未知边界框的两种设置，我们的方法都优于强 DPM 方法。增加一个几何约束 $\delta^{NP}$ 可以改善我们的结果。在全自动设置中，头部排名最高的检测和局部区域定位性能比基准方法好 65％。 $\Delta_{null}= 1$是没有应用几何约束的仅外观情况。虽然细粒度的分类结果没有显示 Δgeometric 和 Δbox 之间的很大差距，但我们可以看到局部区域定位性能的差距。性能差距较小的原因可能在于深层卷积特征对于小的变化和旋转是不变的。  

我们还评估了对边界框和局部区域 selective search region proposals 的 recall。表 3 列出了不同重叠阈值的召回结果。重叠要求为 0.5 时，鸟头部和身体部位的召回率很高，这为根据 region proposals 定位这些部件提供了基础。但是，我们还观察到，当重叠阈值为 0.7 时，头部召回率低于 40％，这表明自下而上的 region proposals 可能是精确局部定位的瓶颈。  

其他可视化显示在图 4 中。我们显示了每个图像的三个检测和局部区域定位，第一列是强 DPM 的输出，第二列是我们的单个局部区域预测的方法，最后一列是我们使用局部优先的方法。我们使用预训练的模型来获得结果。我们还会在图 5 中显示我们方法的一些失败案例。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/Fine-grained/1/5.jpg?raw=true)

### 4.3 Component Analysis  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/Fine-grained/1/6.jpg?raw=true)

为了检验 $\Delta_{geometric}$ 中使用的 $\alpha$ 和 K 不同值的影响，我们进行交叉验证实验。结果显示在图 3 中，我们将图 3 中的 K = 20 固定，并将图 3 中 的 $\alpha = 0.1$ 固定，所有关于以交叉验证方式训练数据的实验，我们将训练数据分成 5 次。如结果所示，端到端的细粒度分类结果对 $\alpha$ 的选择非常敏感，并且 $\alpha = 0.1$ 是没有任何几何约束的 $\Delta_{box}$ 预测的情况。我们必须选择一个小 $\alpha$ 的原因是高斯的 pdf 与我们的局部区域检测器输出的 logistic score 相比较大。另一方面，K 的选择不能太小，当 K 大于 10 时它的选择不是很敏感。  

## 5. Conclusion  

我们已经提出了一种能够进行最先进的细粒度目标识别的联合目标检测和区域定位系统。我们的方法学习探测器和区域模型，并强化区域之间和对象框架之间的几何约束。我们的实验结果表明，即使具有非常强大的特征表示和对象检测系统，通过区域来为具有高语义相似度的类别之间的细粒度区分的困难任务额外地建模对象的姿态也是非常有益的。在今后的工作中，我们将考虑在训练时联合建模对象类别及其各个部分和变形成本的方法。我们还计划探索弱监督设置，在该设置中，我们只会自动发现局部区域并将其模型化为仅来自对象边界框标注的潜在变量。最后，我们将考虑放宽对较小部分的 selective
search，并采用密集窗口采样。

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/Fine-grained/1/7.jpg?raw=true)

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/Fine-grained/1/8.jpg?raw=true)

https://arxiv.org/pdf/1407.3867.pdf